In [2]:
pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Legion\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from typing import Generator
from bs4 import BeautifulSoup, Tag, NavigableString

MAX_LEN = 4096
BLOCK_TAGS = {'p', 'b', 'strong', 'i', 'ul', 'ol', 'div', 'span'}

def split_message(source: str, max_len=MAX_LEN) -> Generator[str, None, None]:
    soup = BeautifulSoup(source, 'html.parser')
    current_fragment = ''
    open_tags = []

    def process_node(node):
        nonlocal current_fragment, open_tags

        if isinstance(node, NavigableString):
            text = str(node)
            idx = 0
            while idx < len(text):
                #вычисляем оставшееся место с закрывающими тегами
                closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
                space_left = max_len - len(current_fragment) - len(closing_tags)
                if space_left <= 0:
                    yield current_fragment + closing_tags
                    #места нет, значит закрываем фрагмент
                    # fragments.append(current_fragment + closing_tags)
                    #начинаем новый фрагмент
                    current_fragment = ''.join(f'<{tag}>' for tag in open_tags)
                    
                    continue 
                chunk = text[idx:idx+space_left]
                current_fragment += chunk
                idx += len(chunk)
        elif isinstance(node, Tag):
            if node.name in BLOCK_TAGS:
                attrs = ''.join(f' {k}="{v}"' for k, v in node.attrs.items())
                start_tag = f"<{node.name}{attrs}>"
                end_tag = f'</{node.name}>'
                
                closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
                potential_length = len(current_fragment) + len(start_tag) + len(end_tag) + len(closing_tags)
                if potential_length > max_len:
                    #места нет, значит закрываем фрагмент
                    # fragments.append(current_fragment + closing_tags)
                    yield (current_fragment + closing_tags)
                    #начинаем новый фрагмент
                    current_fragment = ''.join(f'<{tag}>' for tag in open_tags)
                    
                    closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
                    potential_length = len(current_fragment) + len(start_tag) + len(end_tag) + len(closing_tags)
                    if potential_length > max_len and not node.contents:
                        #если даже пустой тег не помещается делаем исключение
                        raise ValueError("Block Element too long", current_fragment+closing_tags, len(current_fragment+closing_tags))
                    
                current_fragment += start_tag
                open_tags.append(node.name)
        
                for child in node.contents:
                    yield from process_node(child)
                    
                
                closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
                if len(current_fragment) + len(end_tag) + len(closing_tags) > max_len:
                    current_fragment += end_tag
                    open_tags.pop()
                    closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
                    yield (current_fragment + closing_tags)
                    # fragments.append(current_fragment + closing_tags)
                    
                    current_fragment = ''.join(f'<{tag}>' for tag in open_tags)
                    
                else:
                    current_fragment += end_tag
                    open_tags.pop()

            else:
                #добавление неблочных тегов(нельзя разрывать)
                full_tag_str = str(node)
                closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
                if len(full_tag_str) > max_len:
                    raise ValueError("Non block tag text too long", node.name, full_tag_str, len(full_tag_str))
                if len(current_fragment) + len(full_tag_str) + len(closing_tags) > max_len:
                    yield (current_fragment + closing_tags)
                    # fragments.append(current_fragment + closing_tags)
                    current_fragment = ''.join(f'<{tag}>' for tag in open_tags)
                    closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
                    if len(full_tag_str) + len(current_fragment) + len(closing_tags) > max_len:
                        raise ValueError("Nonblock tag too long" ,node.name , current_fragment+full_tag_str+closing_tags, len(current_fragment+ full_tag_str+closing_tags))
                    
                current_fragment += full_tag_str

    for child in soup.contents:
        yield from process_node(child)

    if current_fragment:
        #закрываем все открытые теги 
        closing_tags = ''.join(f'</{tag}>' for tag in reversed(open_tags))
        fragment = (current_fragment + closing_tags)
        yield fragment

    
        


In [38]:
source = open("./source.html", "r").read()
fragments = list(split_message(source, max_len=108))
for idx, fragment in enumerate(fragments, 1):
        print(f"-- fragment #{idx}: {len(fragment)} chars --")
        print(fragment)
        print("\n")

-- fragment #1: 108 chars --
<strong>рџ•’ Some tasks are missing worklogs!</strong>
<mention id="U1024">Justin	Kirvin</mention>
Here is t


-- fragment #2: 107 chars --
he list of tasks that have been in status without worklogs for more than <strong>1h</strong> :arrow_down:




-- fragment #3: 84 chars --
<a href="tg://user?id=3485734953">Talbert Gannaway</a>
<strong>In progress</strong>



-- fragment #4: 108 chars --
<a href="https://mockdata.atlassian.net/browse/ABC-12634"><code>ABC-12634</code></a> Lorem ipsum dolor sit a


-- fragment #5: 65 chars --
met, consectetur adipiscing elit.

<strong>Done</strong>
<i>
</i>


-- fragment #6: 108 chars --
<i><a href="https://mockdata.atlassian.net/browse/ABC-12508"><code>ABC-12508</code></a> Vestibulum pelle</i>


-- fragment #7: 49 chars --
<i>ntesque ullamcorper sapien sed venenatis.
</i>


-- fragment #8: 108 chars --
<i><a href="https://mockdata.atlassian.net/browse/ABC-12587"><code>ABC-12587</code></a> Integer et erat </i>


-- fragmen

In [18]:
with open("./source.html", "r", encoding="utf-8") as f:
    source = f.read()

fragments = list(split_message(source, max_len=108))
for idx, fragment in enumerate(fragments, 1):
    print(f"-- fragment #{idx}: {len(fragment)} chars --")
    print(fragment)
    print("\n")

-- fragment #1: 108 chars --
<strong></strong>
<mention id="U1024">Justin	Kirvin</mention>
Here is the list of tasks that have been in st


-- fragment #2: 68 chars --
atus without worklogs for more than <strong></strong> :arrow_down:




-- fragment #3: 73 chars --
<a href="tg://user?id=3485734953">Talbert Gannaway</a>
<strong></strong>



-- fragment #4: 108 chars --
<a href="https://mockdata.atlassian.net/browse/ABC-12634"><code>ABC-12634</code></a> Lorem ipsum dolor sit a


-- fragment #5: 61 chars --
met, consectetur adipiscing elit.

<strong></strong>
<i></i>



-- fragment #6: 70 chars --
<a href="tg://user?id=604521009">Jilly Paolillo</a>
<strong></strong>



-- fragment #7: 108 chars --
<a href="https://mockdata.atlassian.net/browse/ABC-12255"><code>ABC-12255</code></a> Vestibulum ante ipsum p


-- fragment #8: 34 chars --
rimis in faucibus orci luctus et…



-- fragment #9: 108 chars --
<a href="https://mockdata.atlassian.net/browse/ABC-12365"><code>ABC-12365</code></a> Nunc 

In [31]:
source = open("./source.html", "r").read()
fragments = list(split_message(source, max_len=200))
for idx, fragment in enumerate(fragments, 1):
        print(f"-- fragment #{idx}: {len(fragment)} chars --")
        print(fragment)
        print("\n")

-- fragment #1: 200 chars --
<strong>рџ•’ Some tasks are missing worklogs!</strong>
<mention id="U1024">Justin	Kirvin</mention>
Here is the list of tasks that have been in status without worklogs for more than <strong>1h</strong>


-- fragment #2: 200 chars --
 :arrow_down:

<a href="tg://user?id=3485734953">Talbert Gannaway</a>
<strong>In progress</strong>
<a href="https://mockdata.atlassian.net/browse/ABC-12634"><code>ABC-12634</code></a> Lorem ipsum dolo


-- fragment #3: 200 chars --
r sit amet, consectetur adipiscing elit.

<strong>Done</strong>
<i>
<a href="https://mockdata.atlassian.net/browse/ABC-12508"><code>ABC-12508</code></a> Vestibulum pellentesque ullamcorper sapien </i>


-- fragment #4: 161 chars --
<i>sed venenatis.
<a href="https://mockdata.atlassian.net/browse/ABC-12587"><code>ABC-12587</code></a> Integer et erat mollis, tempor sem a, fringilla est.
</i>



-- fragment #5: 200 chars --
<a href="tg://user?id=604521009">Jilly Paolillo</a>
<strong>Done</strong>
<a href=

In [32]:
source = open("./source.html", "r").read()
fragments = list(split_message(source, max_len=4096))
for idx, fragment in enumerate(fragments, 1):
        print(f"-- fragment #{idx}: {len(fragment)} chars --")
        print(fragment)
        print("\n")

-- fragment #1: 4096 chars --
<strong>рџ•’ Some tasks are missing worklogs!</strong>
<mention id="U1024">Justin	Kirvin</mention>
Here is the list of tasks that have been in status without worklogs for more than <strong>1h</strong> :arrow_down:

<a href="tg://user?id=3485734953">Talbert Gannaway</a>
<strong>In progress</strong>
<a href="https://mockdata.atlassian.net/browse/ABC-12634"><code>ABC-12634</code></a> Lorem ipsum dolor sit amet, consectetur adipiscing elit.

<strong>Done</strong>
<i>
<a href="https://mockdata.atlassian.net/browse/ABC-12508"><code>ABC-12508</code></a> Vestibulum pellentesque ullamcorper sapien sed venenatis.
<a href="https://mockdata.atlassian.net/browse/ABC-12587"><code>ABC-12587</code></a> Integer et erat mollis, tempor sem a, fringilla est.
</i>
<a href="tg://user?id=604521009">Jilly Paolillo</a>
<strong>Done</strong>
<a href="https://mockdata.atlassian.net/browse/ABC-12255"><code>ABC-12255</code></a> Vestibulum ante ipsum primis in faucibus orci luctus etвЂ

In [33]:
source = open("./source.html", "r").read()
fragments = list(split_message(source, max_len=4396))
for idx, fragment in enumerate(fragments, 1):
        print(f"-- fragment #{idx}: {len(fragment)} chars --")
        print(fragment)
        print("\n")

-- fragment #1: 4377 chars --
<strong>рџ•’ Some tasks are missing worklogs!</strong>
<mention id="U1024">Justin	Kirvin</mention>
Here is the list of tasks that have been in status without worklogs for more than <strong>1h</strong> :arrow_down:

<a href="tg://user?id=3485734953">Talbert Gannaway</a>
<strong>In progress</strong>
<a href="https://mockdata.atlassian.net/browse/ABC-12634"><code>ABC-12634</code></a> Lorem ipsum dolor sit amet, consectetur adipiscing elit.

<strong>Done</strong>
<i>
<a href="https://mockdata.atlassian.net/browse/ABC-12508"><code>ABC-12508</code></a> Vestibulum pellentesque ullamcorper sapien sed venenatis.
<a href="https://mockdata.atlassian.net/browse/ABC-12587"><code>ABC-12587</code></a> Integer et erat mollis, tempor sem a, fringilla est.
</i>
<a href="tg://user?id=604521009">Jilly Paolillo</a>
<strong>Done</strong>
<a href="https://mockdata.atlassian.net/browse/ABC-12255"><code>ABC-12255</code></a> Vestibulum ante ipsum primis in faucibus orci luctus etвЂ

In [34]:
source = open("./source.html", "r").read()
fragments = list(split_message(source, max_len=4296))
for idx, fragment in enumerate(fragments, 1):
        print(f"-- fragment #{idx}: {len(fragment)} chars --")
        print(fragment)
        print("\n")

-- fragment #1: 4255 chars --
<strong>рџ•’ Some tasks are missing worklogs!</strong>
<mention id="U1024">Justin	Kirvin</mention>
Here is the list of tasks that have been in status without worklogs for more than <strong>1h</strong> :arrow_down:

<a href="tg://user?id=3485734953">Talbert Gannaway</a>
<strong>In progress</strong>
<a href="https://mockdata.atlassian.net/browse/ABC-12634"><code>ABC-12634</code></a> Lorem ipsum dolor sit amet, consectetur adipiscing elit.

<strong>Done</strong>
<i>
<a href="https://mockdata.atlassian.net/browse/ABC-12508"><code>ABC-12508</code></a> Vestibulum pellentesque ullamcorper sapien sed venenatis.
<a href="https://mockdata.atlassian.net/browse/ABC-12587"><code>ABC-12587</code></a> Integer et erat mollis, tempor sem a, fringilla est.
</i>
<a href="tg://user?id=604521009">Jilly Paolillo</a>
<strong>Done</strong>
<a href="https://mockdata.atlassian.net/browse/ABC-12255"><code>ABC-12255</code></a> Vestibulum ante ipsum primis in faucibus orci luctus etвЂ